<a href="https://colab.research.google.com/github/santoshhub/ML-AI-experiments/blob/master/ML/sentiment_analysis/amazon_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# In this Article, Sentiment Analysis is performed on Amazon Jewelry Dataset.
# Dataset link: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz

#Import the Required Packages
import pandas as pd
import numpy as np
import nltk as nltk
import re 

In [ ]:
#Mount dataset to google drive
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/My Drive/dataset")
!ls

Mounted at /content/drive
amazon_reviews_us_Jewelry_v1_00.tsv  twitter_validation.csv
twitter_training.csv


In [ ]:
#Read the dataset using the pandas
df = pd.read_csv('amazon_reviews_us_Jewelry_v1_00.tsv', sep='\t', header=0, error_bad_lines=False)
df.head(3)

<ipython-input-3-fe307a5c5a17>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('amazon_reviews_us_Jewelry_v1_00.tsv', sep='\t', header=0, error_bad_lines=False)
Skipping line 10437: expected 15 fields, saw 22
Skipping line 10443: expected 15 fields, saw 22
Skipping line 19872: expected 15 fields, saw 22
Skipping line 20055: expected 15 fields, saw 22
Skipping line 20107: expected 15 fields, saw 22
Skipping line 20167: expected 15 fields, saw 22
Skipping line 53858: expected 15 fields, saw 22

Skipping line 72173: expected 15 fields, saw 22
Skipping line 84308: expected 15 fields, saw 22
Skipping line 92156: expected 15 fields, saw 22
Skipping line 97791: expected 15 fields, saw 22
Skipping line 106812: expected 15 fields, saw 22

Skipping line 138899: expected 15 fields, saw 22
Skipping line 145840: expected 15 fields, saw 22
Skipping line 192385: expected 15 fields, saw 22


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,50423057,R135Q3VZ4DQN5N,B00JWXFDMG,657335467,Everbling Purple and Clear Briolette Drop Swar...,Jewelry,5,0.0,0.0,N,Y,Beauties!,so beautiful even tho clearly not high end ......,2015-08-31
1,US,11262325,R2N0QQ6R4T7YRY,B00W5T1H9W,26030170,925 Sterling Silver Finish 6ct Simulated Diamo...,Jewelry,5,0.0,0.0,N,N,Great product.,"Great product.. I got this set for my mother, ...",2015-08-31
2,US,27541121,R3N5JE5Y4T6W5M,B00M2L6KFY,697845240,"Sterling Silver Circle ""Friends Forever"" Infin...",Jewelry,5,0.0,0.0,N,Y,Exactly as pictured and my daughter's friend l...,Exactly as pictured and my daughter's friend l...,2015-08-31


In [ ]:
#We require only the review_body, star_rating columns which describes the reviews, star rating of each review respectively.
df=df[['review_body', 'star_rating']]
df=df.dropna()
df=df.reset_index(drop=True)

In [ ]:
#Remove the Null, missing values and reset the index
df['star_rating']=df['star_rating'].astype(int)
df=df[df['star_rating'] !=3]
df['label']=np.where(df['star_rating']>=4,1,0)
df['star_rating'].value_counts()
df

<ipython-input-6-8c5ece130cd6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label']=np.where(df['star_rating']>=4,1,0)


,review_body,star_rating,label
0,so beautiful even tho clearly not high end ......,5,1
1,"Great product.. I got this set for my mother, ...",5,1
2,Exactly as pictured and my daughter's friend l...,5,1
3,Love it. Fits great. Super comfortable and nea...,5,1
4,Got this as a Mother's Day gift for my Mom and...,5,1
...,...,...,...
1766743,It is nice looking and everything (it is sterl...,4,1
1766744,"my boyfriend bought me this last christmas, an...",4,1
1766745,This is a great way to quickly start learning ...,4,1
1766746,the 14kt gold earrings look remarkable...would...,5,1


In [ ]:
#Remove the Null, missing values and reset the index
df=df.sample(frac=1).reset_index(drop=True)
data=df[df['label']==0][:50000]
data=data.append(df[df['label']==1][:50000])
data=data.reset_index(drop=True)
display(data['label'].value_counts)
display

<ipython-input-9-b9b9e8094de5>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=data.append(df[df['label']==1][:50000])


<bound method IndexOpsMixin.value_counts of 0        0
1        0
2        0
3        0
4        0
        ..
99995    1
99996    1
99997    1
99998    1
99999    1
Name: label, Length: 100000, dtype: int64>

<function IPython.core.display.display(*objs, include=None, exclude=None, metadata=None, transient=None, display_id=None, **kwargs)>

In [ ]:
#Remove the HTML tags and URLs from the reviews.
from bs4 import BeautifulSoup
import requests_oauthlib
import re

def contractions(s):
 s = s.lower()
 s = BeautifulSoup(s).get_text()
 s = re.sub(r"won’t", "will not",s)
 s = re.sub(r"wouldn’t", "would not",s)
 s = re.sub(r"couldn’t", "could not",s)
 s = re.sub(r"\’d", " would",s)
 s = re.sub(r"can\’t", "can not",s)
 s = re.sub(r"n\’t", " not", s)
 s= re.sub(r"\’re", " are", s)
 s = re.sub(r"\’s", " is", s)
 s = re.sub(r"\’ll", " will", s)
 s = re.sub(r"\’t", " not", s)
 s = re.sub(r"\’ve", " have", s)
 s = re.sub(r"\’m", " am", s)
 return s
 
data['pre_process']= data['review_body'].apply(lambda x: contractions(x))

<ipython-input-10-224fae49608f>:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  s = BeautifulSoup(s).get_text()


In [ ]:
import nltk
nltk.download('punkt')
data['pre_process'] = data['pre_process'].apply(lambda x: " ".join([re.sub('[^A-Za-z]+','', x) for x in nltk.word_tokenize(x)]))
data['pre_process'] = data['pre_process'].apply(lambda x: re.sub(' +',' ', x))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Remove the stop words by using the NLTK package
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
data['pre_process'] = data['pre_process'].apply(lambda x: " ".join([x for x in x.split() if x not in stop]))
data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,review_body,star_rating,label,pre_process
0,"I bought four of these necklaces, and three of...",1,0,"bought four necklaces, three broke immediately..."
1,i was very disappointed when the bracelet arri...,2,0,disappointed bracelet arrived tho beautiful or...
2,I bought 3 of these. One for each of my kids b...,1,0,bought 3 these. one kids birthstones within mo...
3,The piece is well made but neckline too high s...,2,0,piece well made neckline high hard comfortable...
4,It change the color as sun as you used. But lo...,1,0,change color sun used. looks great
...,...,...,...,...
99995,This lovely bracelet was a Christmas Gift to m...,5,1,lovely bracelet christmas gift daughter-in law...
99996,Item received pretty quickly. I had to order a...,5,1,item received pretty quickly. order half size ...
99997,Excelent,5,1,excelent
99998,"Amazing detail, excellent quality & incredible...",5,1,"amazing detail, excellent quality & incredible..."


In [ ]:
# Perform lemmatization using the wordnet lemmatizer
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data['pre_process']=data['pre_process'].apply(lambda x: " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(x)]))
data


,review_body,star_rating,label,pre_process
0,"I bought four of these necklaces, and three of...",1,0,bought four necklace three broke immediately h...
1,i was very disappointed when the bracelet arri...,2,0,disappointed bracelet arrived tho beautiful or...
2,I bought 3 of these. One for each of my kids b...,1,0,bought these one kid birthstones within month ...
3,The piece is well made but neckline too high s...,2,0,piece well made neckline high hard comfortable...
4,It change the color as sun as you used. But lo...,1,0,change color sun used look great
...,...,...,...,...
99995,This lovely bracelet was a Christmas Gift to m...,5,1,lovely bracelet christmas gift daughterin law ...
99996,Item received pretty quickly. I had to order a...,5,1,item received pretty quickly order half size s...
99997,Excelent,5,1,excelent
99998,"Amazing detail, excellent quality & incredible...",5,1,amazing detail excellent quality incredible re...


In [ ]:
# Split the Data into Training and Testing sets
from numpy.lib.function_base import vectorize
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data['pre_process'], data['label'], test_size=0.25, random_state=30)
print("Train:", X_train.shape, Y_train.shape, "Test:", X_test.shape, Y_test.shape)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

Train: (75000,) (75000,) Test: (25000,) (25000,)


In [ ]:
# Implementing SVM with sklearn for classification
from sklearn.svm import LinearSVC
clf=LinearSVC(random_state=0)
clf.fit(tf_x_train,Y_train)
y_test_pred=clf.predict(tf_x_test)

In [ ]:
#Implementing SVM with sklearn for classification
from sklearn.metrics import classification_report
report=classification_report(Y_test, y_test_pred,output_dict=True)
print(report)

{'0': {'precision': 0.9140139904110666,
  'recall': 0.9246978371501272,
  'f1-score': 0.9193248745009684,
  'support': 12576},
 '1': {'precision': 0.9228638918302516,
  'recall': 0.9119446233097231,
  'f1-score': 0.91737176632525,
  'support': 12424},
 'accuracy': 0.91836,
 'macro avg': {'precision': 0.918438941120659,
  'recall': 0.9183212302299252,
  'f1-score': 0.9183483204131092,
  'support': 25000},
 'weighted avg': {'precision': 0.9184120374203447,
  'recall': 0.91836,
  'f1-score': 0.9183542578619635,
  'support': 25000}}

In [ ]:
#Logistic Regression
#Logistic regression is implemented using sklearn
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(max_iter=1000,solver='saga')
clf.fit(tf_x_train,Y_train)
y_test_pred=clf.predict(tf_x_test)
from sklearn.metrics import classification_report
report=classification_report(Y_test, y_test_pred,output_dict=True)
print(report)

{'0': {'precision': 0.9140858355604464, 'recall': 0.9246978371501272, 'f1-score': 0.9193612143252431, 'support': 12576}, '1': {'precision': 0.9228701742954879, 'recall': 0.9120251126851255, 'f1-score': 0.9174155938790381, 'support': 12424}, 'accuracy': 0.9184, 'macro avg': {'precision': 0.9184780049279672, 'recall': 0.9183614749176263, 'f1-score': 0.9183884041021406, 'support': 25000}, 'weighted avg': {'precision': 0.9184513005382127, 'recall': 0.9184, 'f1-score': 0.9183943187882971, 'support': 25000}}


In [ ]:
!git clone https://github.com/santoshhub/ML-AI-experiments.git

Cloning into 'ML-AI-experiments'...
